In [1]:
import numpy as np
import matplotlib.pyplot as plt
#import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab 
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx

2024-10-30 20:35:15.360753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 20:35:15.412256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Welcome to JupyROOT 6.30/04


In [2]:
# Set matplotlib default color cycle
new_color_cycle = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#aec7e8',
    '#ffbb78',
    '#98df8a',
    '#ff9896',
    '#c5b0d5',
    '#c49c94',
    '#f7b6d2',
    '#c7c7c7',
    '#dbdb8d',
    '#9edae5'
]

# You can then apply this new color cycle to your matplotlib plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_color_cycle)

In [3]:
L1AD_rate = 1000
target_rate = 10

### Test standard scalers

### Test ONNX

In [7]:
training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_2", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    "train_data_scheme": 
    "topo2A_train+overlap", 
    "pt_normalization_type": 
    "global_division", 
    "L1AD_rate": 1000
}
model_version=0
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='HLT', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
ef.convert_to_onnx(training_info=training_info, model_version=model_version, object_type='L1', save_dir='./trained_models/multiple_trainings/trial_2/onnx')
datasets, data_info = ef.load_and_preprocess(**data_info)
datasets = ef.compare_tf_with_onnx(datasets=datasets, training_info=training_info, model_version=0, onnx_path='./trained_models/multiple_trainings/trial_2/onnx')

Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:26.715362: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:26.715559: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:27.230193: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:27.230453: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX HLT_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/HLT_AE_0.onnx


Could not search for non-variable resources. Concrete function internal representation may have changed.
2024-10-30 15:45:29.812788: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:29.812992: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-10-30 15:45:30.345443: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-10-30 15:45:30.345661: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


ONNX L1_AE model saved to: ./trained_models/multiple_trainings/trial_2/onnx/L1_AE_0.onnx
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5


In [8]:
for tag, data_dict in datasets.items():
    print(f'{tag}:')
    for key, value in data_dict.items():
        print(f'    {key}: {value.shape}')

A14N23LO:
    HLT_data: (10000, 48)
    L1_data: (10000, 48)
    passHLT: (10000,)
    passL1: (10000,)
    topo2A_AD_scores: (10000,)
    weights: (10000,)
    L1Seeded: (10000,)
    HLT_model_outputs: (10000, 48)
    L1_model_outputs: (10000, 48)
    HLT_AD_scores: (10000,)
    L1_AD_scores: (10000,)
    ONNX_HLT_model_outputs: (10000, 48)
    ONNX_L1_model_outputs: (10000, 48)
    ONNX_HLT_AD_scores: (10000,)
    ONNX_L1_AD_scores: (10000,)
HAHMggfZdZd2l2nu:
    HLT_data: (70000, 48)
    L1_data: (70000, 48)
    passHLT: (70000,)
    passL1: (70000,)
    topo2A_AD_scores: (70000,)
    weights: (70000,)
    L1Seeded: (70000,)
    HLT_model_outputs: (70000, 48)
    L1_model_outputs: (70000, 48)
    HLT_AD_scores: (70000,)
    L1_AD_scores: (70000,)
    ONNX_HLT_model_outputs: (70000, 48)
    ONNX_L1_model_outputs: (70000, 48)
    ONNX_HLT_AD_scores: (70000,)
    ONNX_L1_AD_scores: (70000,)
HHbbttHadHad:
    HLT_data: (100000, 48)
    L1_data: (100000, 48)
    passHLT: (100000,)
    pa

In [9]:
print(datasets['EB_test']['HLT_AD_scores'][0:5])
print()
print(datasets['EB_test']['ONNX_HLT_AD_scores'][0:5])

[1.62544931 1.92602879 1.33561259 0.         1.06108748]

[1.62544931 1.92602879 1.33561259 0.         1.06108748]


In [ ]:
# Success! These AD scores are the same.

In [4]:
# Rerun evals for trial 1 with correct test data (require test data to have passed L1)

data_info = {
    "train_data_scheme": "topo2A_train+L1noalg_HLTall", 
     "pt_normalization_type": "global_division", 
     "L1AD_rate": L1AD_rate
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_1", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'_fixed_test_data/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
evals phase 1 of 2 initiated.
phase 1: starting evals of model 0...


2024-10-26 07:01:40.362610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 07:01:40.377635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 07:01:40.377859: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

HLTAD threshold: 9.286888124404394
HLTAD threshold: 4.561022975888879
phase 1: starting evals of model 1...
HLTAD threshold: 8.733034961640545
HLTAD threshold: 4.752744620679564
phase 1: starting evals of model 2...
HLTAD threshold: 5.498442664832609
HLTAD threshold: 4.022619764534142
phase 1: starting evals of model 3...
HLTAD threshold: 8.63785676118642
HLTAD threshold: 3.9006244314475182
phase 1: starting evals of model 4...
HLTAD threshold: 8.7621613195813
HLTAD threshold: 3.740239170525538
phase 1: starting evals of model 5...
HLTAD threshold: 8.733262628992186
HLTAD threshold: 3.655159783666013
phase 1: starting evals of model 6...
HLTAD threshold: 9.330566657464805
HLTAD threshold: 4.258327901457669
phase 1: starting evals of model 7...
HLTAD threshold: 8.839285093274306
HLTAD threshold: 3.8142383037773233
phase 1: starting evals of model 8...
HLTAD threshold: 8.61077566014482
HLTAD threshold: 3.8344466905367085
phase 1: starting evals of model 9...
HLTAD threshold: 8.8791002359

In [5]:
# Rerun evals for trial 2 with correct test data (require test data to have passed L1)

data_info = {
    "train_data_scheme": "topo2A_train+overlap", 
     "pt_normalization_type": "global_division", 
     "L1AD_rate": L1AD_rate
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_2", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'_fixed_test_data/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
evals phase 1 of 2 initiated.
phase 1: starting evals of model 0...
HLTAD threshold: 7.069826961453727
HLTAD threshold: 5.324225057374288
phase 1: starting evals of model 1...
HLTAD threshold: 7.03006376295605
HLTAD threshold: 5.06360493238033
phase 1: starting evals of model 2...
HLTAD threshold: 7.042411270027907
HLTAD threshold: 5.1820545487871
phase 1: s

In [6]:
# Rerun evals for trial 3 with correct test data (require test data to have passed L1)

data_info = {
    "train_data_scheme": "L1noalg_HLTall", 
     "pt_normalization_type": "global_division", 
     "L1AD_rate": L1AD_rate
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_3", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'_fixed_test_data/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
evals phase 1 of 2 initiated.
phase 1: starting evals of model 0...
HLTAD threshold: 7.656267000375177
HLTAD threshold: 3.754804002275115
phase 1: starting evals of model 1...
HLTAD threshold: 7.700342502939914
HLTAD threshold: 3.7171524179343565
phase 1: starting evals of model 2...
HLTAD threshold: 6.605279562879243
HLTAD threshold: 4.038791468866988
phase

In [ ]:
# Rerun evals for trial 4 with correct test data (require test data to have passed L1)

data_info = {
    "train_data_scheme": "topo2A_train+overlap", 
     "pt_normalization_type": "StandardScaler", 
     "L1AD_rate": L1AD_rate
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_4", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'_fixed_test_data/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
evals phase 1 of 2 initiated.
phase 1: starting evals of model 0...
HLTAD threshold: 1.4599743448750353
HLTAD threshold: 2.028436779010568
phase 1: starting evals of model 1...
HLTAD threshold: 1.4986319312620693
HLTAD threshold: 2.0434982870312806
phase 1: starting evals of model 2...
HLTAD threshold: 1.5118034763737196
HLTAD threshold: 2.2659821200397765
p

In [ ]:
# Rerun evals for trial 5 with correct test data (require test data to have passed L1)

data_info = {
    "train_data_scheme": "topo2A_train+overlap", 
     "pt_normalization_type": "per_event", 
     "L1AD_rate": L1AD_rate
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_5", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10
}

ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'_fixed_test_data/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

### Main scheme

In [5]:
data_info = {
    "train_data_scheme": "topo2A_train+overlap", 
    "pt_normalization_type": "StandardScaler", 
    "L1AD_rate": 1000
}

training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_8", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

datasets, data_info = ef.load_and_preprocess(**data_info)
training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)

Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
Booting up... initializing trainings of 10 models

starting training model 0...

Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 13: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 19: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.

Epoch 25: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.

Ep

In [6]:
ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate
)

powering on... preparing to run evals
Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
evals phase 1 of 2 initiated.
phase 1: starting evals of model 0...
HLTAD threshold: 29.844683214341877
HLTAD threshold: 34.37642446134512
phase 1: starting evals of model 1...
HLTAD threshold: 29.84450039083802
HLTAD threshold: 34.247196032071706
phase 1: starting evals of model 2...
HLTAD threshold: 27.04230594704984
HLTAD threshold: 34.2360536488412
phase

Info in <TEfficiency::TEfficiency>: given histograms are filled with weights
Info in <TROOT::TEfficiency::SetUseWeightedEvents>: Handle weighted events for computing efficiency
Warning in <TEfficiency::GetEfficiencyErrorLow>: frequentist confidence intervals for weights are only supported by the normal approximation
Info in <TEfficiency::GetEfficiencyErrorLow>: setting statistic option to kFNormal
Info in <TCanvas::Print>: png file ./trained_models/multiple_trainings/trial_8/plots/HLT_0_pileup_efficiency.png has been created
Info in <TEfficiency::TEfficiency>: given histograms are filled with weights
Info in <TROOT::TEfficiency::SetUseWeightedEvents>: Handle weighted events for computing efficiency
Warning in <TEfficiency::GetEfficiencyErrorLow>: frequentist confidence intervals for weights are only supported by the normal approximation
Info in <TEfficiency::GetEfficiencyErrorLow>: setting statistic option to kFNormal
Info in <TCanvas::Print>: png file ./trained_models/multiple_trainin

In [ ]:
# ef.process_multiple_models(
#     training_info=training_info,
#     data_info=data_info,
#     plots_path=training_info['save_path']+'/plots',
#     target_rate=target_rate,
#     L1AD_rate=L1AD_rate
# )

Changes to make:
- different L1AD rates
- l1All ROC
- l1All efficiency distribution

### save data with AD scores

In [4]:
# Save version with AD scores
training_info = {
    "save_path": "./trained_models/multiple_trainings/trial_2", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10,
    "training_weights": True
}

data_info = {
    "train_data_scheme": 
    "topo2A_train+overlap", 
    "pt_normalization_type": 
    "global_division", 
    "L1AD_rate": 1000
}

datasets, data_info = ef.load_and_preprocess(**data_info)
ef.save_datasets_with_AD_scores(datasets, training_info=training_info, save_dir='/eos/home-m/mmcohen/ntuples/dataset_with_AD_scores')

Loaded A14N23LO from ./h5_ntuples/A14N23LO.h5
Loaded EB from ./h5_ntuples/EB.h5
Loaded EB_test2 from ./h5_ntuples/EB_test2.h5
Loaded EB_train from ./h5_ntuples/EB_train.h5
Loaded HAHMggfZdZd2l2nu from ./h5_ntuples/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from ./h5_ntuples/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from ./h5_ntuples/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from ./h5_ntuples/ZZ4lep.h5
Loaded Zprime2EJs from ./h5_ntuples/Zprime2EJs.h5
Loaded jjJZ1 from ./h5_ntuples/jjJZ1.h5
Loaded jjJZ2 from ./h5_ntuples/jjJZ2.h5
Loaded jjJZ4 from ./h5_ntuples/jjJZ4.h5
Loaded qqa from ./h5_ntuples/qqa.h5
15912/15912 [==============================] - 24s 1ms/step
Saved A14N23LO to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/A14N23LO.h5
Saved HAHMggfZdZd2l2nu to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/HAHMggfZdZd2l2nu.h5
Saved HHbbttHadHad to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/HHbbttHadHad.h5
Saved ZZ4lep to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/ZZ4l

In [5]:
datasets = ef.load_subdicts_from_h5('/eos/home-m/mmcohen/ntuples/dataset_with_AD_scores')

for tag, data_dict in datasets.items():
    print(f'{tag}:')
    for key, value in data_dict.items():
        print(f'    {key}: {value.shape}')

Loaded A14N23LO from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/A14N23LO.h5
Loaded EB_test from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/EB_test.h5
Loaded EB_train from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/EB_train.h5
Loaded EB_val from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/EB_val.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/HHbbttHadHad.h5
Loaded ZZ4lep from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/ZZ4lep.h5
Loaded Zprime2EJs from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/Zprime2EJs.h5
Loaded jjJZ1 from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/jjJZ1.h5
Loaded jjJZ2 from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/jjJZ2.h5
Loaded jjJZ4 from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/jjJZ4.h5
Loaded qqa from /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/qqa.h5
A1